In [22]:
import tensorflow as tf
print(tf.__version__)

2.15.0


# Problem Statement

# Data Preprocessing

In [23]:
import tensorflow as tf
from tensorflow import keras

train = tf.keras.utils.image_dataset_from_directory('CNN Data/train', labels = 'inferred')
val = tf.keras.utils.image_dataset_from_directory('CNN Data/val', labels = 'inferred')

Found 9999 files belonging to 10 classes.
Found 2000 files belonging to 10 classes.


# Deep Learning Model: CNN

In [24]:
types = ['Amphibia', 'Animalia', 'Arachnida', 'Aves', 'Fungi', 'Insecta', 'Mammalia', 'Mollusca', 'Plantae', 'Reptilia']

In [25]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Rescaling(1./255),
    tf.keras.layers.RandomFlip("horizontal_and_vertical"),
    tf.keras.layers.RandomRotation(0.2),

    tf.keras.layers.Conv2D(32, 3, padding = 'same', activation = 'relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(64, 3, padding = 'same', activation = 'relu'),
    tf.keras.layers.MaxPooling2D(),
    #tf.keras.layers.Conv2D(128, 3, padding = 'same', activation = 'relu'),
    #tf.keras.layers.MaxPooling2D(),

    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation=tf.nn.relu),
    tf.keras.layers.Dense(512, activation=tf.nn.relu),
    #tf.keras.layers.Dense(512, activation=tf.nn.relu),
    tf.keras.layers.Dense(256, activation=tf.nn.relu),
    tf.keras.layers.Dense(256, activation=tf.nn.relu),
    #tf.keras.layers.Dense(256, activation=tf.nn.relu),
    tf.keras.layers.Dense(128, activation=tf.nn.relu),
    tf.keras.layers.Dense(128, activation=tf.nn.relu),
    #tf.keras.layers.Dense(128, activation=tf.nn.relu),
    tf.keras.layers.Dense(10)#, activation=tf.nn.softmax)
])

In [26]:
class myCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs={}):
    if(logs.get('val_accuracy') > 0.75):
      print("Cancelling training.")
      self.model.stop_training = True

callbacks = myCallback()

In [27]:
model.compile(optimizer=tf.optimizers.Adam(),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

Best cycle:
Epoch 28/60
313/313 [==============================] - 130s 414ms/step - loss: 1.8414 - accuracy: 0.3435 - val_loss: 1.8969 - val_accuracy: 0.3300

In [28]:
model.fit(train, validation_data=val, epochs=60, callbacks=[callbacks])

Epoch 1/60


313/313 [==============================] - 118s 371ms/step - loss: 2.2982 - accuracy: 0.1126 - val_loss: 2.2611 - val_accuracy: 0.1470
Epoch 2/60
313/313 [==============================] - 175s 559ms/step - loss: 2.2674 - accuracy: 0.1404 - val_loss: 2.2186 - val_accuracy: 0.1510
Epoch 3/60
313/313 [==============================] - 115s 368ms/step - loss: 2.2285 - accuracy: 0.1608 - val_loss: 2.2048 - val_accuracy: 0.1760
Epoch 4/60
313/313 [==============================] - 123s 394ms/step - loss: 2.2147 - accuracy: 0.1709 - val_loss: 2.2065 - val_accuracy: 0.1730
Epoch 5/60
313/313 [==============================] - 119s 380ms/step - loss: 2.2095 - accuracy: 0.1663 - val_loss: 2.2053 - val_accuracy: 0.1750
Epoch 6/60
 20/313 [>.............................] - ETA: 1:46 - loss: 2.2166 - accuracy: 0.1562

# Evaluation of Model

In [ ]:
model.evaluate(val)

63/63 [==============================] - 3s 49ms/step - loss: 2.1981 - accuracy: 0.2265


[2.198082447052002, 0.226500004529953]

# Hyperparameter Tuning